# RAG for ColaborEJA

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from transformers import BitsAndBytesConfig
import torch
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
#from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

nb_4bit_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    "rhaymison/Mistral-portuguese-luana-7b",
    quantization_config=nb_4bit_config,
    device_map={"": 0}
)
tokenizer = AutoTokenizer.from_pretrained("rhaymison/Mistral-portuguese-luana-7b")
model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): Mist

In [3]:
pipe = pipeline("text-generation", 
                model=model, 
                device_map='auto',
                tokenizer=tokenizer, 
                use_cache = True,
                do_sample=True,
                temperature=0.75,
                top_p=0.95,
                repetition_penalty = 1.2,
                max_new_tokens=2000)

llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cuda:0
C:\Users\Gui\AppData\Local\Temp\ipykernel_19504\610688578.py:12: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [34]:
# Prompt template
PROMPT_TEMPLATE = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.
Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

# Prompt template
PROMPT_SPECIALIZED = """
You are a highly sophisticated artificial intelligence specialized in digital literacy,
with a focus on Youth and Adult Education, assisting teachers in creating lesson plans
that meet the needs of their students.
Use the contextual excerpts provided to answer the question and develop a lesson plan to assist the teacher.
Be respectful and pay attention to Brazilian Portuguese spelling.
If you do not know the answer, simply say that you do not know. DO NOT MAKE ANYTHING UP.
Use four sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

# Prompt template
PROMPT_TABLE = """
You are a highly sophisticated artificial intelligence specialized in digital literacy,
with a focus on Youth and Adult Education, assisting teachers in creating lesson plans
that meet the needs of their students.
Use the contextual excerpts provided to answer the question.
Be respectful and pay attention to Brazilian Portuguese spelling.
If you do not know the answer, simply say that you do not know. DO NOT MAKE ANYTHING UP.
Use four sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""


PROMPT_TABLE_BR = """
  Você é uma inteligência artificial altamente sofisticada, especializada em
  letramento digital, com foco na Educação de Jovens e Adultos (EJA),
  auxiliando professores na criação de planos de aula que atendam às necessidades de seus alunos.
  Use os trechos de contexto fornecidos para responder à pergunta.
  Seja respeitoso e atente à ortografia do português do Brasil.
  Se não souber a resposta, diga apenas que não sabe. NÃO INVENTE NADA.
  Use no máximo três frases e mantenha a resposta concisa.
  Pergunta: {question}
  Contexto: {context}
  Answer:"""

#Analise formulario
PROMPT_FORMS = """
  Você é uma inteligência artificial altamente sofisticada, especializada em
  letramento digital, com foco na Educação de Jovens e Adultos (EJA)
  Por meio do formulário passado como contexto, realize a tarefa passada. 
  NÃO INVENTE NADA e mantenha as respostas conscisas.
  Tarefa:{question}
  Contexto: {context}
  Answer:
"""

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

def get_text_splitter():
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1500,
                                               chunk_overlap = 500,
                                               length_function=len,
                                               separators=["\n\n", "\n", " "])
    return text_splitter

def get_embeddings_function():
    embeddings = HuggingFaceEmbeddings(
        model_name ="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings
embedding_function = get_embeddings_function()



In [6]:
import uuid
from langchain.document_loaders import CSVLoader

def create_vectorstore(chunks, embedding_function, vectorstore_path):

    # Create a list of unique ids for each document based on the content
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]

    # Ensure that only unique docs with unique ids are kept
    unique_ids_set = set()
    unique_chunks = []
    unique_ids_list = []
    
    for chunk, id in zip(chunks, ids):
        if id not in unique_ids_set:
            unique_ids_set.add(id)
            unique_chunks.append(chunk)
            unique_ids_list.append(id)

    # Create a new Chroma database from the documents
    vectorstore = Chroma.from_documents(documents=unique_chunks,
                                        ids=unique_ids_list,
                                        embedding=embedding_function,
                                        persist_directory = vectorstore_path)
    return vectorstore

# Create retriever and get relevant chunks
def get_retriver(question, vectorstore):
    retriever = vectorstore.as_retriever(search_type="similarity")
    relevant_chunks = retriever.invoke(question)
    return relevant_chunks

def generate(question, vectorstore, prompt = PROMPT_TEMPLATE):
    relevant_chunks = get_retriver(question = question, vectorstore = vectorstore)
    # Concatenate context text
    context_text = "\n\n---\n\n".join([doc.page_content for doc in relevant_chunks])

    # Create prompt
    prompt_template = ChatPromptTemplate.from_template(prompt)
    prompt = prompt_template.format(context=context_text, 
                                    question = question)
    response = llm.invoke(prompt)
    return response

def load_pdf(pdf_file, vector_path):
    loader = PyPDFLoader(pdf_file, extraction_mode="layout")
    pages = loader.load()
    text_splitter = get_text_splitter()
    chunks = text_splitter.split_documents(pages)
    vectorstore = create_vectorstore(
        chunks=chunks,
        embedding_function=embedding_function,
        vectorstore_path=vector_path
    )
    return vectorstore

def load_CSV(csv_file, vector_path):
    loader = CSVLoader(csv_file, encoding="utf-8")
    document = loader.load()
    text_splitter = get_text_splitter()
    chunks = text_splitter.split_documents(document)
    vectorstore = create_vectorstore(
        chunks=chunks,
        embedding_function=embedding_function,
        vectorstore_path=vector_path
    )
    return vectorstore

def load_answer(answer):
    prev_answer = embedding_function.embed_query(answer)

In [7]:
vectorstore_qprinc = load_pdf("diretivas\REquestoes_principais.pdf", "vectorstore_table")
vectorstore_qalt = load_pdf("diretivas\questoes_alternativas.pdf", "vectorstore_alt")
vectorstore_plan = load_pdf("diretivas\Exemplo de plano de aula.pdf", "vectorstore_plan")

c:\Users\Gui\Documents\USP\TCC\mistral_luana\venv\Lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [8]:
vector_forms = load_CSV("Cópia do Formulário (respostas) - Respostas ao formulário 1.csv", "vectorstore_csv")

In [9]:
# Deletar dados DB

def del_vector(vector_path):
    vectorstore = Chroma(
        persist_directory=vector_path,
        embedding_function=embedding_function
    )

    # Recupera todos os IDs dos documentos armazenados
    all_ids = vectorstore.get()['ids']

    # Apaga os documentos usando os IDs
    vectorstore.delete(ids=all_ids)
    print("Vector data deleted")

In [ ]:
answer = generate("""Identifique as principais 
                  dificuldades que um jovem com autismo moderado pode apresentar nas 
                  áreas de comunicação, coordenação, cooperação e consciência. 
                  Para cada área, liste as questões orientadoras, os atributos de 
                  qualidade envolvidos (como eficácia ou eficiência), e os componentes do 
                  MHP que podem estar comprometidos (como sensores, memória, 
                  processadores cognitivos, motores, etc.). Use os dados da fonte como contexto para sua resposta.""",
                  vectorstore_qprinc)
print(answer)
#print(answer.split("Answer:")[-1])

In [ ]:
query = """
Por meio do formulário passado como contexto, identifique explicitamnete TODAS
as dificuldades ou necessidades especiais apresentadas na turma.
  """
answer = generate(query, vector_forms, PROMPT_FORMS)
print(answer)

Human: 
  Você é uma inteligência artificial altamente sofisticada, especializada em
  letramento digital, com foco na Educação de Jovens e Adultos (EJA)
  Por meio do formulário passado como contexto, realize a tarefa passada. 
  NÃO INVENTE NADA e mantenha as respostas conscisas.
  Tarefa:
Por meio do formulário passado como contexto, identifique explicitamnete TODAS
as dificuldades ou necessidades especiais apresentadas na turma.
  
  Contexto: Carimbo de data/hora: 12/05/2025 16:39:01
Pontuação: 
Idade:: 30
Sexo:: Masculino
Estado Civil:: Solteiro
Número de filhos:: Três
Ocupação atual (profissão):: Vendedor
Horário de trabalho: Dia todo
Renda familiar mensal (valor aproximado):: Entre 1 e 2 salários mínimos
Úlima série concluída:: 9° ano do fundamental
Ano de conclusão:: 2000
Motivo pelo qual interrompeu os estudos:: Trabalho
Já estudou na EJA alguma vez?: Não
O que motivou você a estudar?
(Marque todas as opções aplicáveis): Melhoria de oportunidades de emprego., Crescimento pess

# tentando uma outra abordagem
## Usando langgraph

In [ ]:

# from langgraph.graph import START, StateGraph
# from typing_extensions import List, TypedDict
# from langchain_core.documents import Document

# prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
# # Define state for application
# class State(TypedDict):
#     question: str
#     context: List[Document]
#     answer: str

#     # Define application steps
# def retrieve(state: State):
#     retrieved_docs = vectorstore.similarity_search(state["question"])
#     return {"context": retrieved_docs}


# def generate(state: State):
#     docs_content = "\n\n".join(doc.page_content for doc in state["context"])
#     messages = prompt_template.invoke({"question": state["question"], "context": docs_content})
#     response = llm.invoke(messages)
#     return {"answer": response}

# # Compile application and test
# graph_builder = StateGraph(State).add_sequence([retrieve, generate])
# graph_builder.add_edge(START, "retrieve")
# graph = graph_builder.compile()

In [ ]:
# response = graph.invoke({"question": """Quais são as questões de acessibilidade que um jovem com autismo pode encontrar? 
#                     Quais as possíveis soluções?
#                     Faça um novo plano de aula baseado nas questões anteriores e usando o Plano de Aula como exemplo.
#                     """})
# print(response["answer"])


Human: 
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer
the question. If you don't know the answer, say that you
don't know. DON'T MAKE UP ANYTHING.
Use three sentences maximum and keep the answer concise.
Question: Quais são as questões de acessibilidade que um jovem com autismo pode encontrar? 
                    Quais as possíveis soluções?
                    Faça um novo plano de aula baseado nas questões anteriores e usando o Plano de Aula como exemplo.
                     
Context:  
Answer: As principais questões de acessibilidade para alguém com autismo incluem dificuldade em se adaptar ao ambiente social, problemas de comunicação e dificuldades sensoriais. Algumas das soluções podem envolver ter estruturas predictáveis, treinamento sensorial e aprendizagem de habilidades sociais especiais. Um novo plano de aula pode ser criado considerando esses fatores e incorporando técnicas como modelagem visual, rotina e exercíc